# 🖊️ Pen Plotter — Clean Quickstart (GRBL, Safe)

A modular, beginner-friendly workflow for controlling a GRBL-based pen plotter with Python.

- Connect and configure your device
- Manually home the arm (move to Arduino for origin)
- Calibrate pen contact
- Plot a custom SVG
- Safely close the connection

All logic is organized in `penplot_helper.py`. Follow the steps below!

## 🏠 Manual Homing

Move the pen arm gently towards the Arduino to set the origin (x=0, y=0).
This step ensures accurate positioning before plotting.

## ⚙️ Initialization

Set up your pen plotter and connect to GRBL below.
Make sure your device is on the correct port.

In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from penplot_helper import Config, GRBL
import numpy as np
import matplotlib.pyplot as plt

# Connect
cfg = Config(
    port="/dev/tty.usbserial-A50285BI",
    x_max=300.0, y_max=245.0,
    s_down=90, s_up=40,
)
grbl = GRBL(cfg).connect()

# Warm up status and print it
grbl.ensure_wpos()
grbl.status()


{'raw': '<Idle|MPos:0.000,0.000,0.000|Bf:15,128|FS:0,0|Ov:100,100,100>',
 'state': 'Idle',
 'wpos': (0.0, 0.0, 0.0)}

## Test rectangle sweep to set paper size

In [52]:
# Define bed and do a quick perimeter check (pen up)
grbl.pen_up(step=0.1)
#grbl.set_bed(300.0, 225.0)
grbl.sweep_rect(cfg.x_max, cfg.y_max)

# Go to center (absolute work coords)
grbl.goto_abs(cfg.x_max/2.0, cfg.y_max/2.0)
grbl.goto_abs(  0.0,   0.0)

grbl.pen_down(step=0.1)

Goto absolute: (150.000, 122.500)
Goto absolute: (0.000, 0.000)
Goto absolute: (0.000, 0.000)


In [53]:
grbl.status()

{'raw': '<Idle|MPos:0.000,0.000,0.000|Bf:15,128|FS:0,90|WCO:0.000,0.000,0.000>',
 'state': 'Idle',
 'wpos': (0.0, 0.0, 0.0)}

## 📐 Select Project Area

Interactively choose the bottom left and top right corners of your project area. The surface compensation will use these bounds for calibration.

In [54]:
from penplot_widgets import show_area_and_compensation_widget
get_area_and_comp = show_area_and_compensation_widget(grbl)  # Interactively set area and compensation (motor updates on slider change)

## 🖼️ Plot a Custom SVG

Scale and center your SVG, then plot it with compensated pen movement.

## Setup conveniences

In [33]:
import penplot_widgets as ppw
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
saved = getattr(ppw, "_PPW_STATE", {}).get(key, None)
grbl.set_compensation_from_widget(saved)

# from penplot_helper import Config, GRBL
# from pattern import Pattern, Renderer, Line, Polyline
# import penplot_widgets as ppw

# # Load saved widget state (compensation + pots) — keep all checks here
# key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
# saved = getattr(ppw, "_PPW_STATE", {}).get(key, None)

# if saved:
#     grbl.set_compensation_from_widget(saved)
#     #pots = Pot.list_from_widget_state(saved)
#     BL, TR = saved['corners']['BL'], saved['corners']['TR']
#     rect = (float(BL['x']), float(BL['y']), float(TR['x']), float(TR['y']))  # (x0,y0,x1,y1)
# else:
#     # Fallback: full bed, no pots
#     pots = []
#     rect = (0.0, 0.0, cfg.x_max, cfg.y_max)

# # Convenience
# cx = 0.5*(rect[0]+rect[2]); cy = 0.5*(rect[1]+rect[3])


In [ ]:
from pattern import Pattern, Renderer, Polyline, Circle, Line
import math, random

# Home
grbl.pen_up(step=0.1)
grbl.goto_abs(0.0, 0.0)
random.seed(42)

# Minimal pattern: 2 circles + 1 line + 1 polyline
p = Pattern()
p.add(
    Circle((40.0, 40.0), 12.0, start_deg=0,   sweep_deg=360, feed_draw=3000),
    Circle((80.0, 40.0), 18.0, start_deg=180, sweep_deg=180, feed_draw=6000),  # semicircle
    Line((20.0, 80.0), (100.0, 80.0), feed_draw=3500),
    Polyline([(20.0, 20.0), (50.0, 25.0), (80.0, 22.0)], feed_draw=7000),
)

# Renderer (your params)
r = Renderer(grbl,
             z_mode="centroid",
             z_threshold=0.1,
             settle_down_s=0.04,
             settle_up_s=0.04,
             z_step=0.1,
             z_step_delay=0.00,
             flush_every=300,
             feed_travel=10000,
             lift_delta=0.2)

# Run with preprocessing
r.run(p,
      start_xy=(0.0, 0.0),  # for nn optimization
      optimize='nn',
      combine={'join_tol_mm': 0.1},
      resample={'max_dev_mm': 0.1, 'max_seg_mm': None})

Goto absolute: (0.000, 0.000)


TypeError: Circle.__init__() got an unexpected keyword argument 'seg_len_mm'

In [32]:
p.preview()

'POLYLINE n=3239 (50.0, 50.0)->(148.83594115158022, 49.976804499565496)\nPOLYLINE n=3239 (148.83594115158022, 49.976804499565496)->(50.0, 50.0)'

In [ ]:
# Add a highly oversampled pattern with 0.1 mm increments
for i in range(200):  # 200 lines, each 0.1 mm apart for high density
    x_start = 0
    x_end = 10
    y = i * 0.1  # 0.1 mm increments
    p.add([(x_start, y), (x_end, y)])  # Add horizontal line

## Run experiments

### Example

In [ ]:
# # Geometry: center + radius
# x0, y0, x1, y1 = rect
# cx, cy = 0.5*(x0+x1), 0.5*(y0+y1)
# r = 0.30 * min((x1-x0), (y1-y0))   # 30% of the smaller rect side

# # Plan: one compensated circle (slight extra pressure)
# pat = Pattern().add(
#     Circle(center=(cx, cy), radius=r, pos_offset=-0.12, settle_s=0.10, ccw=True, feed_draw=9000)
# )

# # Preview and run
# plot_plan((cfg.x_max, cfg.y_max), rect, pots, pat, step=50.0, figsize=(8.5, 5.5), title="Center Circle");
# Renderer(grbl).run(pat)
# grbl.goto_abs(0.0, 0.0)

### Circle

In [ ]:
from penplot_experiments import ex1_circles_through_pile

# Plot only (do not execute):
pat = ex1_circles_through_pile(
    cfg=cfg,
    grbl=grbl,
    rect=rect,                # (x0, y0, x1, y1) from the widget
    pots=pots,                # list of Pot objects (for plotting legend/markers)

    # ---- Parameters (defaults shown) ----
    n_angles=3,              # number of directions around the pile
    start_lead_deg=10.0,      # start ~10° before the pile so each circle passes through it
    pos_off=-0.20,            # extra down-pressure (added to compensated pen pos); clipped to [0,1]
    settle_s=0.10,            # dwell after pen_set before XY motion
    feed_draw=15000,           # drawing feed (mm/min)
    safety_mm=0.6,            # keep circle inside rect by this clearance
    pile_xy=None,             # (x, y) of paint pile; None → rect center

    # ---- Output controls ----
    plot=True,                # show preview figure
    execute=True,            # send to plotter if True

    # ---- Figure tweaks (passed to plot_plan) ----
    figure_kwargs=dict(step=50.0, figsize=(9.2, 6.0), title="Maximal circles through center pile")
)

### Tringles

In [ ]:
# --- Experiment: maximal triangles from center pile (preview or run) ---
from penplot_experiments import ex2_triangles_from_pile

# Preview only
pat = ex2_triangles_from_pile(
    cfg=cfg,
    grbl=grbl,
    rect=rect,                 # (x0,y0,x1,y1) from widget
    pots=pots,                 # for plotting markers

    # ---- Parameters (feel free to tweak) ----
    n_angles=40,               # number of directions around the pile
    half_apex_deg=35.0,        # half of the tip angle at the apex; larger = fatter triangles
    lead_mm=10.0,               # start this far before the apex along the first edge (scoop paint)
    pos_off=-0.30,             # extra down-pressure
    settle_s=0.1,             # servo settle after Z set
    feed_draw=15000,            # drawing feed
    safety_mm=0.6,             # rectangle clearance
    pile_xy=None,              # None → use rect center

    # ---- Output controls ----
    plot=True,
    execute=False,             # set True to run on the plotter
    figure_kwargs=dict(step=50.0, figsize=(9.2, 6.0), title="Maximal triangles from center pile")
)

# To execute, just set execute=True above.
grbl.goto_abs(0.0, 0.0)


### Field

In [ ]:
from penplot_experiments import ex4c_curl_push_multi
import penplot_widgets as ppw

# --- Load rect & pots from the widget cache (used by the function too; here only to size params nicely) ---
key  = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = getattr(ppw, "_PPW_STATE", {}).get(key, {})
pots  = state.get("pots", [])
n_pots = len(pots)

# --- Choose which pots to use (e.g., the first 2). Adjust as needed. ---
pot_indices = [0,1,2,3]#list(range(min(2, n_pots)))  # use first two pots if available

# --- Per-pot agent counts: list[int] matching pot_indices (or shorter → padded with last) ---
agents_per_pot = [30, 10,10,10]  # example: more strokes from pot 0 than pot 1

# --- Run (plot only by default) ---
pat = ex4c_curl_push_multi(
    cfg=cfg, grbl=grbl,
    rect=None,           # auto from widget cache
    pots=None,           # auto from widget cache
    pot_indices=pot_indices,

    agents_per_pot=agents_per_pot,  # list OR int
    steps=80,
    step_mm=2.0,

    # Flow presets (smooth ribbons)
    noise_scale=8.0,
    curl_strength=2.5,
    drift_gain=0.25,
    jitter_deg=0.0,

    # Avoid hitting other pots; stop at boundary
    repulse_radius_mm=30.0,
    repulse_gain=2.0,
    boundary_safety=0.0,
    stop_at_boundary=True,

    # Ensure each stroke passes through its pot first
    lead_in_mm=5.0,
    lead_out_mm=5.0,

    # Motion / Z
    z_mode="threshold",
    pos_off=-0.10,
    settle_s=0.12,
    feed_draw=9000,
    min_seg_mm=9.0,

    # Progressive tracing (0→p%) per agent (set to False to disable)
    progressive=False,
    progressive_increments=6,   # 10%, 20%, …, 100%
    progressive_reverse=False,    # alternate direction each pass

    # Preview / execute
    random_ini=51,
    plot=True,
    execute=False,
    figure_kwargs=dict(step=50.0, figsize=(9.8, 6.4),
                       title="Curl-Noise Push — multi-pot (per-pot counts & colors)")
)

# To execute on the plotter, set execute=True in the call above.
# Optional: return home afterwards
# grbl.goto_abs(0.0, 0.0)

### Lines Einstein

In [ ]:
# --- Einstein (tongue) → grayscale → concentric circles nudged by darkness ---
# Prereqs in notebook: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import numpy as np, math, requests
from io import BytesIO
from PIL import Image
import penplot_widgets as ppw
from pattern import Pattern, Polyline, Renderer, plot_plan

# 1) Load Einstein-from-Commons (RGB) → grayscale in [0,1]
r = requests.get(IMG_URL, timeout=60, headers={"User-Agent": "Mozilla/5.0 (compatible; penplotter/1.0)"})
r.raise_for_status()
img = np.asarray(Image.open(BytesIO(r.content)).convert("RGB"))
gray = (0.2126*img[...,0] + 0.7152*img[...,1] + 0.0722*img[...,2]).astype(np.float32) / 255.0
Himg, Wimg = gray.shape

# 2) Get current drawing rectangle from the calibration widget
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
W, H = x1 - x0, y1 - y0

# 3) Parameters
origin_corner     = "BL"          # "BL", "BR", "TL", "TR"
n_rings           = 150           # number of concentric circles
samples_per_ring  = 1000           # angle samples per circle
dr_mm             = 2*min(W, H) / (n_rings + 2)   # radial spacing
offset_mm         = 3.0           # radial nudge for black pixels; white ≈ 0
pos_off           = -0.20
settle_s          = 0.08
feed_draw         = 11000
execute           = False         # set True to run on plotter

# 4) Origin at a rectangle corner
if origin_corner == "BL":
    ox, oy = x0, y0
elif origin_corner == "BR":
    ox, oy = x1, y0
elif origin_corner == "TL":
    ox, oy = x0, y1
else:  # "TR"
    ox, oy = x1, y1

# 5) Bilinear sampler of grayscale at world (x,y) mapped to image indices
def g_at(x, y):
    u = (x - x0) / max(1e-9, W)
    v = 1.0 - (y - y0) / max(1e-9, H)
    xi = np.clip(u * (Wimg - 1), 0, Wimg - 1)
    yi = np.clip(v * (Himg - 1), 0, Himg - 1)
    x0i = int(np.floor(xi)); x1i = min(Wimg - 1, x0i + 1)
    y0i = int(np.floor(yi)); y1i = min(Himg - 1, y0i + 1)
    wx = xi - x0i; wy = yi - y0i
    a = gray[y0i, x0i]; b = gray[y0i, x1i]; c = gray[y1i, x0i]; d = gray[y1i, x1i]
    return float((a*(1-wx) + b*wx)*(1-wy) + (c*(1-wx) + d*wx)*wy)

# 6) Clip a polyline to the rectangle by dropping outside points and splitting
def clip_poly_to_rect(points, rect):
    x0,y0,x1,y1 = rect
    out, cur = [], []
    for (x,y) in points:
        inside = (x0 <= x <= x1) and (y0 <= y <= y1)
        if inside:
            cur.append((x,y))
        else:
            if len(cur) >= 2:
                out.append(cur)
            cur = []
    if len(cur) >= 2:
        out.append(cur)
    return out

# 7) Build concentric, nudged circles and clip to rect
pat = Pattern()
for k in range(1, n_rings+1):
    r_base = k * dr_mm
    pts = []
    for j in range(samples_per_ring+1):
        th = (2*math.pi) * (j / samples_per_ring)
        # sample grayscale at base point
        sx, sy = ox + r_base*math.cos(th), oy + r_base*math.sin(th)
        g = g_at(sx, sy)               # 0..1
        r = r_base + offset_mm * (1.0 - g)
        x, y = ox + r*math.cos(th), oy + r*math.sin(th)
        pts.append((x, y))
    # clip → possibly several arcs per ring
    for seg in clip_poly_to_rect(pts, (x0,y0,x1,y1)):
        poly = Polyline(pts=seg, z_mode="threshold", z_threshold=0.2, pos_offset=pos_off, settle_s=settle_s,
                        min_seg_mm=1.0, max_seg_mm=1e9, feed_draw=feed_draw)
        setattr(poly, "_plot_color", "#000000")
        pat.add(poly)

# 8) Preview & (optionally) execute
plot_plan((cfg.x_max, cfg.y_max), (x0,y0,x1,y1), [], pat,
          step=50.0, figsize=(9.6,6.2), title=f"Einstein circles (origin={origin_corner})")

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0.0, 0.0)


### Lines Nina Stefan

In [ ]:
# --- stefanNina.png → grayscale → sine-stripes with image-driven amplitude & frequency ---
# Prereqs in notebook: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.
# Put stefanNina.png next to your notebook (or change IMG_PATH below).

import os, math, numpy as np, penplot_widgets as ppw
from PIL import Image
from pattern import Pattern, Polyline, Renderer, plot_plan

# 1) Load image (RGB) → grayscale in [0,1]
IMG_PATH = "stefanNina.png"   # <-- change if needed
assert os.path.exists(IMG_PATH), f"Image not found: {IMG_PATH}"
img = np.asarray(Image.open(IMG_PATH).convert("RGB"))
gray = (0.2126*img[...,0] + 0.7152*img[...,1] + 0.0722*img[...,2]).astype(np.float32) / 255.0
Himg, Wimg = gray.shape

# 2) Get current drawing rectangle from the calibration widget
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
W, H = x1 - x0, y1 - y0

# 3) Parameters (tweak freely)
n_rows            = 60            # how many sine lines (rows)
row_margin_mm     = 4.0            # top/bottom margin inside the rectangle
dx_mm             = 0.03            # sampling step along X (smaller = smoother curves)
phase_jitter_deg  = np.pi*4            # per-row starting phase jitter

# Amplitude & frequency mapping from grayscale (g=0 black → strong; g=1 white → weak)
A_base_mm         = 0.0            # base amplitude
A_gain_mm         = 2.0            # extra amplitude * (1 - g)
f_base_cpm        = 1.0           # base frequency (cycles per mm)
f_gain_cpm        = 0.0          # added frequency * (1 - g)  (e.g. up to ~0.02 cyc/mm)

# Motion / pen
pos_off           = -0.20
settle_s          = 0.25
feed_draw         = 3000
execute           = True        # set True to run on plotter

# 4) Bilinear sampler of grayscale at world (x,y) mapped to image indices
def g_at(x, y):
    # map world → [0,1] in image (letterbox/center-fit into rect width & height)
    u = (x - x0) / max(1e-9, W)
    v = 1.0 - (y - y0) / max(1e-9, H)
    xi = np.clip(u * (Wimg - 1), 0, Wimg - 1)
    yi = np.clip(v * (Himg - 1), 0, Himg - 1)
    x0i = int(np.floor(xi)); x1i = min(Wimg - 1, x0i + 1)
    y0i = int(np.floor(yi)); y1i = min(Himg - 1, y0i + 1)
    wx = xi - x0i; wy = yi - y0i
    a = gray[y0i, x0i]; b = gray[y0i, x1i]; c = gray[y1i, x0i]; d = gray[y1i, x1i]
    return float((a*(1-wx) + b*wx)*(1-wy) + (c*(1-wx) + d*wx)*wy)

# 5) Clip a polyline to the rectangle by dropping outside points and splitting
def clip_poly_to_rect(points, rect):
    rx0,ry0,rx1,ry1 = rect
    out, cur = [], []
    for (x,y) in points:
        inside = (rx0 <= x <= rx1) and (ry0 <= y <= ry1)
        if inside:
            cur.append((x,y))
        else:
            if len(cur) >= 2:
                out.append(cur)
            cur = []
    if len(cur) >= 2:
        out.append(cur)
    return out

# 6) Build sine rows with image-driven amplitude & frequency
pat = Pattern()

usable_H = max(0.0, H - 2*row_margin_mm)
if n_rows < 1 or usable_H <= 0:
    raise ValueError("n_rows must be ≥1 and H must be larger than 2*row_margin_mm.")

row_step = usable_H / (n_rows - 1 if n_rows > 1 else 1)
rng = np.random.default_rng(2025)

for r in range(n_rows):
    y_base = y0 + row_margin_mm + r*row_step
    # start phase (optionally jitter per row)
    phi = math.radians(phase_jitter_deg) * float(rng.uniform(-1.0, 1.0)) if phase_jitter_deg > 0 else 0.0

    pts = []
    x = x0
    while x <= x1 + 1e-6:
        # sample grayscale along the *baseline* (x, y_base)
        g = g_at(x, y_base)  # 0 (dark) .. 1 (light)
        # local amplitude & frequency (cycles/mm)
        A = A_base_mm + A_gain_mm * (1.0 - g)
        f = f_base_cpm + f_gain_cpm * (1.0 - g)
        # y offset by sine with current phase
        y = y_base + A * math.sin(phi)
        pts.append((x, y))
        # advance phase according to local frequency (Δφ = 2π f Δx)
        phi += 2.0*math.pi * f * dx_mm
        x += dx_mm

    # clip to rect (sine can overshoot vertically)
    for seg in clip_poly_to_rect(pts, (x0,y0,x1,y1)):
        poly = Polyline(
            pts=seg,
            z_mode="midpoints",      # smooth & fast
            pos_offset=pos_off,
            settle_s=settle_s,
            min_seg_mm=0.8,
            max_seg_mm=1e9,
            feed_draw=feed_draw,
        )
        setattr(poly, "_plot_color", "#000000")
        pat.add(poly)

# 7) Preview & (optionally) execute
plot_plan((cfg.x_max, cfg.y_max), (x0,y0,x1,y1), [], pat,
          step=50.0, figsize=(9.6,6.2),
          title="stefanNina → sine stripes (amplitude & frequency from darkness)")

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0.0, 0.0)


### Lines 2

In [ ]:
# --- Boundary-chord fill with nonuniform boundary sampling (3D-ish density illusion) ---
# Prereqs: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import math, numpy as np, penplot_widgets as ppw
from pattern import Pattern, Polyline, Renderer, plot_plan

# 1) Load calibrated rectangle + program Z surface map (for height compensation)
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
corn = state["corners"]
grbl.set_surface_map(float(corn["BL"]["h"]), float(corn["BR"]["h"]),
                     float(corn["TL"]["h"]), float(corn["TR"]["h"]))

# 2) Parameters (tweak freely)
W, H       = (x1 - x0), (y1 - y0)
margin_mm  = 6.0
execute    = False

# Boundary sampling
n_points   = 100                 # total boundary points to place
dens_amp1  = 0.80; freq1 = 2     # low-frequency clustering
dens_amp2  = 0.35; freq2 = 5     # higher-frequency clustering
phase1     = math.radians(20.0)
phase2     = math.radians(-60.0)

# Optional localized “hotspots” along perimeter fraction t∈[0,1) (wrap-around Gaussian bumps)
hotspots   = [(0.12, 0.08, 0.9), (0.62, 0.05, 0.6)]  # (center_t, sigma, gain)

# Chord mapping (who connects to whom)
families   = [
    dict(k_base=0.50, k_amp=0.08, phase=math.radians(0.0)),   # across with mild modulation
    dict(k_base=0.33, k_amp=0.06, phase=math.radians(40.0)),  # second family adds weave
]
# Motion
pos_off    = -0.14
settle_s   = 0.05
feed_draw  = 12000

# 3) Perimeter helpers: s∈[0,P) ↔ (x,y), with P = 2(W+H)
P = 2.0*(W + H)

def s_to_xy(s):
    """Map perimeter arclength s→(x,y), CCW from (x0,y0) along bottom→right→top→left."""
    s = float(s) % P
    if s < W:                         # bottom: (x0+s, y0)
        return (x0 + s, y0)
    s -= W
    if s < H:                         # right: (x1, y0+s)
        return (x1, y0 + s)
    s -= H
    if s < W:                         # top: (x1-s, y1)
        return (x1 - s, y1)
    s -= W                            # left: (x0, y1-s)
    return (x0, y1 - s)

def xy_to_s(x, y):
    """Inverse map (approx, for bookkeeping)."""
    if abs(y - y0) < 1e-6 and x0 <= x <= x1:      # bottom
        return (x - x0) % P
    if abs(x - x1) < 1e-6 and y0 <= y <= y1:      # right
        return (W + (y - y0)) % P
    if abs(y - y1) < 1e-6 and x0 <= x <= x1:      # top
        return (W + H + (x1 - x)) % P
    if abs(x - x0) < 1e-6 and y0 <= y <= y1:      # left
        return (W + H + W + (y1 - y)) % P
    # If slightly inside boundary (numerics), snap to nearest edge:
    dists = [
        (abs(y - y0), (x0 + np.clip(x - x0, 0, W), y0)),
        (abs(x - x1), (x1, y0 + np.clip(y - y0, 0, H))),
        (abs(y - y1), (x1 - np.clip(x1 - x, 0, W), y1)),
        (abs(x - x0), (x0, y1 - np.clip(y1 - y, 0, H))),
    ]
    _, (xx, yy) = min(dists, key=lambda t: t[0])
    return xy_to_s(xx, yy)

# 4) Density along perimeter: w(t) > 0 for t∈[0,1)
def wrap_dist(a, b):
    """Shortest distance on unit circle between parameters a,b in [0,1)."""
    d = abs((a - b) % 1.0)
    return min(d, 1.0 - d)

def weight_t(t):
    w = 1.0 + dens_amp1*math.cos(2*math.pi*freq1*t + phase1) \
            + dens_amp2*math.cos(2*math.pi*freq2*t + phase2)
    for (tc, sigma, gain) in hotspots:
        d = wrap_dist(t, tc)
        w += gain * math.exp(-0.5*(d/sigma)**2)
    return max(1e-3, w)

# Build CDF for inverse-transform sampling
M = 8192
ts = np.linspace(0.0, 1.0, M, endpoint=False)
ws = np.array([weight_t(t) for t in ts], dtype=np.float64)
cdf = np.cumsum(ws); cdf /= cdf[-1]

# Sample perimeter positions (sorted to give a nice ordering)
u  = np.linspace(0.0, 1.0, n_points, endpoint=False) + (1.0/n_points)*0.5
t_samples = np.interp(u, cdf, ts)
s_samples = (t_samples * P)  # arclengths
s_samples.sort()

# 5) Build chord pairs with smoothly varying offset (k_base ± k_amp*sin)
pairs = set()
for fam in families:
    k_base = fam['k_base']    # fraction of N
    k_amp  = fam['k_amp']
    kphi   = fam['phase']
    for i, s in enumerate(s_samples):
        k = int(round(k_base*n_points + k_amp*n_points * math.sin(2*math.pi*(s/P) + kphi)))
        j = (i + k) % n_points
        if i == j: continue
        a, b = (i, j) if i < j else (j, i)
        pairs.add((a, b))

# 6) Convert to segments (skip those fully outside due to margin clamp)
def clamp_to_rect(x, y, eps=1e-9):
    X = min(x1, max(x0, x))
    Y = min(y1, max(y0, y))
    return (X, Y)

segments = []
for i, j in sorted(pairs):
    ax, ay = s_to_xy(s_samples[i]); bx, by = s_to_xy(s_samples[j])
    ax, ay = clamp_to_rect(ax, ay);  bx, by = clamp_to_rect(bx, by)
    # prune degenerate or identical points
    if (ax == bx and ay == by): continue
    segments.append(((ax, ay), (bx, by)))

# 7) Pattern
pat = Pattern()
for (A, B) in segments:
    poly = Polyline(
        pts=[A, B],
        z_mode="midpoints",
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.0,
        max_seg_mm=1e12,
        feed_draw=feed_draw,
    )
    setattr(poly, "_plot_color", "#000000")
    pat.add(poly)

# 8) Preview & (optional) execute
title = (f"Boundary-chord fill • N={n_points} • families={len(families)} "
         f"• hotspots={len(hotspots)}")
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat,
          step=50.0, figsize=(9.6, 6.2), title=title)

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0,0)


### Shapes

In [ ]:
# --- Hairball worms: self-avoiding worm-like chains from the center with Gaussian heading jitter ---
# Prereqs: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import math, random, numpy as np, penplot_widgets as ppw
from pattern import Pattern, Polyline, Renderer, plot_plan

# 1) Load calibrated rectangle + program Z surface map
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
corn = state["corners"]
grbl.set_surface_map(float(corn["BL"]["h"]), float(corn["BR"]["h"]),
                     float(corn["TL"]["h"]), float(corn["TR"]["h"]))

# 2) Parameters (tune freely)
W, H = (x1 - x0), (y1 - y0)
cx, cy = (x0 + W*0.5, y0 + H*0.5)

n_chains        = 50            # number of worms
steps_per_chain = 100           # segments per worm
step_mm         = 1.0           # segment length
jitter_deg      = 3.0           # Gaussian heading jitter (std dev in degrees)
repel_strength  = 300.0          # how strongly worms avoid existing segments
edge_strength   = 100.0          # repulsion from rectangle edges
min_clear_mm    = 0.2          # minimum clearance to any segment; if violated, we steer harder
ignore_tail_k   = 0            # ignore last k segments of current chain for avoidance (prevents self-stall)
seed            = 1337          # RNG seed for reproducibility (None for random)

# Motion
pos_off   = -0.2
settle_s  = 0.05
feed_draw = 3000
execute   = True               # set True to run on plotter

rng = random.Random(seed)

# 3) Spatial index for existing segments (fast local queries)
cell = max(2.5, step_mm*2.2)
grid = {}  # (ix,iy) -> list of segment indices
segments = []  # global list of segments: [(x0,y0,x1,y1)]
owners   = []  # owner chain index per segment

def _cell_ixy(x, y):
    return (int((x - x0) // cell), int((y - y0) // cell))

def _cells_for_seg(x0s, y0s, x1s, y1s):
    ix0, iy0 = _cell_ixy(min(x0s,x1s), min(y0s,y1s))
    ix1, iy1 = _cell_ixy(max(x0s,x1s), max(y0s,y1s))
    for i in range(ix0-1, ix1+2):
        for j in range(iy0-1, iy1+2):
            yield (i,j)

def _add_segment(x0s, y0s, x1s, y1s, owner):
    idx = len(segments)
    segments.append((x0s,y0s,x1s,y1s))
    owners.append(owner)
    for key in _cells_for_seg(x0s,y0s,x1s,y1s):
        grid.setdefault(key, []).append(idx)

def _neighbors(x, y):
    ix, iy = _cell_ixy(x, y)
    for i in range(ix-1, ix+2):
        for j in range(iy-1, iy+2):
            for idx in grid.get((i,j), ()):
                yield idx

# Geometry helpers
def _closest_point_on_seg(px, py, x0s, y0s, x1s, y1s):
    vx, vy = (x1s-x0s, y1s-y0s)
    L2 = vx*vx + vy*vy
    if L2 <= 1e-12:
        return x0s, y0s, math.hypot(px-x0s, py-y0s)
    t = ((px-x0s)*vx + (py-y0s)*vy) / L2
    t = 0.0 if t < 0.0 else (1.0 if t > 1.0 else t)
    qx, qy = (x0s + t*vx, y0s + t*vy)
    return qx, qy, math.hypot(px-qx, py-qy)

def _edge_repulse(px, py):
    # inward normals from each edge with 1/d^2 decay
    eps = 1e-6
    dxL = max(eps, px - x0); dxR = max(eps, x1 - px)
    dyB = max(eps, py - y0); dyT = max(eps, y1 - py)
    rx = (1.0/dxL**2) - (1.0/dxR**2)
    ry = (1.0/dyB**2) - (1.0/dyT**2)
    return rx, ry

# 4) Grow worms
chains = []  # list of polylines: each is [(x,y), ...]
sigma = math.radians(jitter_deg)
k_rep = repel_strength
k_edge = edge_strength

# Distribute initial headings around the circle for nicer coverage
init_headings = [2*math.pi * (i / n_chains) for i in range(n_chains)]
rng.shuffle(init_headings)

for ci in range(n_chains):
    pts = [(cx, cy)]
    theta = init_headings[ci] + rng.uniform(-0.2, 0.2)  # small per-chain variation

    for s in range(steps_per_chain):
        xh, yh = pts[-1]

        # --- compute repulsion from nearby segments (all chains + older parts of this chain)
        rx, ry = 0.0, 0.0
        for idx in _neighbors(xh, yh):
            # skip very recent segments from the same chain to reduce self-stall
            if owners[idx] == ci and idx >= len(segments) - ignore_tail_k:
                continue
            x0s,y0s,x1s,y1s = segments[idx]
            qx, qy, d = _closest_point_on_seg(xh, yh, x0s, y0s, x1s, y1s)
            if d < 1e-6:  # on top → push random
                ang = rng.random()*2*math.pi
                rx += math.cos(ang); ry += math.sin(ang)
                continue
            # repulsion vector away from closest point with 1/d^2 kernel
            w = 1.0 / (d*d)
            rx += (xh - qx) * w
            ry += (yh - qy) * w

        # boundary repulsion
        ex, ey = _edge_repulse(xh, yh)
        rx += k_edge * ex
        ry += k_edge * ey

        # normalize and steer (angle domain), + Gaussian jitter
        if rx != 0.0 or ry != 0.0:
            phi = math.atan2(ry, rx)
            # steer a fraction toward repulsion direction
            # wrap smallest angle difference
            dtheta = ((phi - theta + math.pi) % (2*math.pi)) - math.pi
            theta += (k_rep * 0.002) * dtheta  # small steering gain

        theta += rng.gauss(0.0, sigma)

        # propose next point
        nx = xh + step_mm * math.cos(theta)
        ny = yh + step_mm * math.sin(theta)

        # reflect if we would step outside the rectangle
        bounced = False
        if nx < x0:
            nx = x0 + (x0 - nx); theta = math.pi - theta; bounced = True
        elif nx > x1:
            nx = x1 - (nx - x1); theta = math.pi - theta; bounced = True
        if ny < y0:
            ny = y0 + (y0 - ny); theta = -theta; bounced = True
        elif ny > y1:
            ny = y1 - (ny - y1); theta = -theta; bounced = True

        # if too close to any neighbor after moving, nudge heading more and retry (one attempt)
        too_close = False
        for idx in _neighbors(nx, ny):
            if owners[idx] == ci and idx >= len(segments) - ignore_tail_k:
                continue
            x0s,y0s,x1s,y1s = segments[idx]
            _, _, d = _closest_point_on_seg(nx, ny, x0s, y0s, x1s, y1s)
            if d < min_clear_mm:
                too_close = True
                break
        if too_close:
            theta += rng.gauss(0.0, 2*sigma)
            nx = xh + step_mm * math.cos(theta)
            ny = yh + step_mm * math.sin(theta)
            # if still outside, clamp again
            nx = min(max(nx, x0), x1)
            ny = min(max(ny, y0), y1)

        pts.append((nx, ny))
        _add_segment(xh, yh, nx, ny, owner=ci)

        # optional early stop if the worm gets stuck near boundary corners
        if bounced and s > 12 and math.hypot(nx - xh, ny - yh) < 0.1:
            break

    chains.append(pts)

# 5) Build pattern
pat = Pattern()
for pts in chains:
    if len(pts) < 2: continue
    poly = Polyline(
        pts=pts,
        z_mode="midpoints",     # one Z per segment midpoint → smooth & fast
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.4,
        max_seg_mm=1e12,
        feed_draw=feed_draw,
    )
    setattr(poly, "_plot_color", "#000000")
    pat.add(poly)

# 6) Preview & (optionally) execute
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat,
          step=50.0, figsize=(9.6, 6.2), title="Hairball worms (self-avoiding with jitter)")

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0.0, 0.0)


### planets

In [ ]:
# --- Gravity-fan: rays deflected by randomly placed "planets" (circles) with smooth arclength integration ---
# Prereqs in notebook: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import math, random, numpy as np, penplot_widgets as ppw
from pattern import Pattern, Polyline, Renderer, plot_plan

# 1) Load calibrated rectangle (+ program Z surface map for consistent Z)
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
corn = state["corners"]
grbl.set_surface_map(float(corn["BL"]["h"]), float(corn["BR"]["h"]),
                     float(corn["TL"]["h"]), float(corn["TR"]["h"]))

# 2) Parameters (you tuned many of these; keep them, plus smoother integration controls below)
W, H = (x1 - x0), (y1 - y0)
rng_seed           = 1113
n_planets          = 20
r_min_frac, r_max_frac = 0.003, 0.03  # radius as fraction of min(W,H)
min_sep_frac       = 0.05             # allow overlaps if desired
circle_samples     = 90

origin_corner      = "BL"            # "BL","BR","TL","TR"
fan_center_deg     = math.degrees(math.atan2((y0+H*0.5)-(y0 if origin_corner[0]=='B' else y1),
                                             (x0+W*0.5)-(x0 if origin_corner[1]=='L' else x1)))
fan_span_deg       = 120.0
n_rays             = 120

# --- Integration / motion parameters (smooth arclength + speed ramp) ---
step_mm     = 0.5     # arclength per emitted vertex (smaller = smoother)
substeps    = 4       # micro-steps per emitted vertex (stability without oversampling)
v0_mm       = 2    # starting speed magnitude
cruise_mm   = 2    # target speed after ramp
ramp_steps  = 60      # how many emitted vertices to ramp v0 → cruise
max_steps   = 620     # cap *emitted vertices* per ray

soft_mm            = 5   # gravitational softening
G_eff              = 10   # overall gravity strength
accel_clip         = 10*0.5  # clamp |a| per physics second^2
hit_margin_mm      = -0.5   # stop if inside (r + margin)
launch_inset_mm    = 2.0   # start slightly inside the rectangle

# Motion / pen
pos_off   = -0.20
settle_s  = 0.25
execute   = False

rng = random.Random(rng_seed)

# 3) Helpers
def corner_xy(tag):
    if tag == "BL": return (x0+launch_inset_mm, y0+launch_inset_mm)
    if tag == "BR": return (x1-launch_inset_mm, y0+launch_inset_mm)
    if tag == "TL": return (x0+launch_inset_mm, y1-launch_inset_mm)
    return (x1-launch_inset_mm, y1-launch_inset_mm)

def circle_poly(cx, cy, r, n=circle_samples):
    return [(cx + r*math.cos(2*math.pi*i/n), cy + r*math.sin(2*math.pi*i/n)) for i in range(n+1)]

# 4) Scatter planets
R0 = min(W, H)
min_sep = min_sep_frac * R0
planets = []  # (cx,cy,r,mass)
tries = 0
while len(planets) < n_planets and tries < 5000:
    tries += 1
    r = rng.uniform(r_min_frac*R0, r_max_frac*R0)
    cx = rng.uniform(x0 + r + 2.0, x1 - r - 2.0)
    cy = rng.uniform(y0 + r + 2.0, y1 - r - 2.0)
    ok = True
    for (px, py, pr, _) in planets:
        if math.hypot(cx - px, cy - py) < max(min_sep, 0.65*(r + pr)):
            ok = False; break
    if ok:
        m = (r*r)  # mass ∝ area
        planets.append((cx, cy, r, m))

# 5) Planet outlines
pat = Pattern()
for (cx, cy, r, _) in planets:
    poly = Polyline(
        pts=circle_poly(cx, cy, r),
        z_mode="threshold",
        feed_draw=300,
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.5,
        max_seg_mm=1e9,
    )
    setattr(poly, "_plot_color", "#000000")
    pat.add(poly)

# 6) Gravity field + utilities
def gravity_accel(x, y):
    ax = ay = 0.0
    for (cx, cy, r, m) in planets:
        dx, dy = (cx - x), (cy - y)
        d2 = dx*dx + dy*dy + soft_mm*soft_mm
        inv_d = 1.0 / math.sqrt(d2)
        inv_d3 = inv_d * inv_d * inv_d
        a = G_eff * m * inv_d3
        ax += a * dx
        ay += a * dy
    mag = math.hypot(ax, ay)
    if mag > accel_clip:
        ax *= accel_clip / mag
        ay *= accel_clip / mag
    return ax, ay

def inside_rect(x, y):
    return (x0 <= x <= x1) and (y0 <= y <= y1)

def hits_planet(x, y):
    for (cx, cy, r, _) in planets:
        if math.hypot(x - cx, y - cy) <= (r + hit_margin_mm):
            return True
    return False

# 7) Launch fan (velocity-Verlet with constant arclength + speed ramp)
ox, oy = corner_xy(origin_corner)
theta0 = math.radians(fan_center_deg)
thetas = np.linspace(theta0 - math.radians(fan_span_deg)*0.5,
                     theta0 + math.radians(fan_span_deg)*0.5, n_rays)

for th in thetas:
    x, y = ox, oy
    vx, vy = v0_mm*math.cos(th), v0_mm*math.sin(th)
    ax, ay = gravity_accel(x, y)
    pts = [(x, y)]

    for step in range(max_steps):
        # ramp target speed
        if step < ramp_steps:
            v_target = v0_mm + (cruise_mm - v0_mm) * (step / float(ramp_steps))
        else:
            v_target = cruise_mm

        ds  = step_mm
        dsm = ds / float(substeps)
        broke = False

        for _ in range(substeps):
            # dt chosen so that |v|*dt = dsm  → constant arclength per micro-step
            dt = dsm / max(1e-9, v_target)

            # half-kick
            vx += 0.5 * ax * dt
            vy += 0.5 * ay * dt
            # renormalize to target speed
            vm = math.hypot(vx, vy) or 1.0
            vx *= (v_target / vm)
            vy *= (v_target / vm)
            # drift
            x += vx * dt
            y += vy * dt
            if (not inside_rect(x, y)) or hits_planet(x, y):
                broke = True
                break
            # accel at new position + half-kick
            ax, ay = gravity_accel(x, y)
            vx += 0.5 * ax * dt
            vy += 0.5 * ay * dt

        if broke:
            break
        pts.append((x, y))

    if len(pts) >= 2:
        poly = Polyline(
            pts=pts,
            z_mode="threshold",
            z_threshold=0.8,
            pos_offset=pos_off,
            settle_s=settle_s,
            min_seg_mm=0.6,
            max_seg_mm=1e9,
            feed_draw=4000,
        )
        setattr(poly, "_plot_color", "#000000")
        pat.add(poly)

# 8) Preview & (optionally) execute
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat,
          step=50.0, figsize=(9.6, 6.2),
          title=f"Gravity fan from {origin_corner} (span {fan_span_deg}°, {n_rays} rays)")

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0.0, 0.0)


### Planets with equal horizontal lines

In [ ]:
# --- Gravity-flow “scanlines”: 120 horizontal rays entering from the left, deflected by planets ---
# Prereqs: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import math, random, numpy as np, penplot_widgets as ppw
from pattern import Pattern, Polyline, Renderer, plot_plan

# 1) Load calibrated rectangle (+ program Z surface map)
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
corn = state["corners"]
grbl.set_surface_map(float(corn["BL"]["h"]), float(corn["BR"]["h"]),
                     float(corn["TL"]["h"]), float(corn["TR"]["h"]))

# 2) Parameters
W, H = (x1 - x0), (y1 - y0)
rng_seed           = 1113
n_planets          = 20
r_min_frac, r_max_frac = 0.005, 0.06
min_sep_frac       = 0.0
circle_samples     = 120

n_rays             = 120         # number of horizontal rays
launch_inset_mm    = 0.0         # inset from left edge
y_inset_mm         = 0.0         # inset from top/bottom for seeding
start_jitter_mm    = 0.0         # optional vertical jitter (0 = evenly spaced)

# Integration (smooth arclength + ramped speed)
step_mm     = 0.45
substeps    = 4
v0_mm       = 1
cruise_mm   = 1
ramp_steps  = 50
max_steps   = 800

soft_mm     = 2.0
G_eff       = 0.5
accel_clip  = 0.5
hit_margin_mm = 0.5

# Motion / pen
pos_off   = -0.20
settle_s  = 0.25
feed_draw = 6000
execute   = False

rng = random.Random(rng_seed)

# 3) Helpers
def circle_poly(cx, cy, r, n=circle_samples):
    return [(cx + r*math.cos(2*math.pi*i/n), cy + r*math.sin(2*math.pi*i/n)) for i in range(n+1)]

# 4) Scatter planets
R0 = min(W, H)
min_sep = min_sep_frac * R0
planets = []  # (cx,cy,r,mass)
tries = 0
while len(planets) < n_planets and tries < 5000:
    tries += 1
    r = rng.uniform(r_min_frac*R0, r_max_frac*R0)
    cx = rng.uniform(x0 + r + 2.0, x1 - r - 2.0)
    cy = rng.uniform(y0 + r + 2.0, y1 - r - 2.0)
    ok = True
    for (px, py, pr, _) in planets:
        if math.hypot(cx - px, cy - py) < max(min_sep, 0.65*(r + pr)):
            ok = False; break
    if ok:
        m = (r*r)
        planets.append((cx, cy, r, m))

# 5) Pattern + planet outlines
pat = Pattern()
for (cx, cy, r, _) in planets:
    poly = Polyline(
        pts=circle_poly(cx, cy, r),
        z_mode="midpoints",
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.5,
        max_seg_mm=1e9,
        feed_draw=1000,
    )
    setattr(poly, "_plot_color", "#000000")
    pat.add(poly)

# 6) Gravity field
def gravity_accel(x, y):
    ax = ay = 0.0
    for (cx, cy, r, m) in planets:
        dx, dy = (cx - x), (cy - y)
        d2 = dx*dx + dy*dy + soft_mm*soft_mm
        inv_d = 1.0 / math.sqrt(d2)
        inv_d3 = inv_d * inv_d * inv_d
        a = G_eff * m * inv_d3
        ax += a * dx
        ay += a * dy
    mag = math.hypot(ax, ay)
    if mag > accel_clip:
        ax *= accel_clip / mag
        ay *= accel_clip / mag
    return ax, ay

def inside_rect(x, y): return (x0 <= x <= x1) and (y0 <= y <= y1)
def hits_planet(x, y):
    for (cx, cy, r, _) in planets:
        if math.hypot(x - cx, y - cy) <= (r + hit_margin_mm):
            return True
    return False

# 7) Launch horizontal rays from left edge (evenly spaced vertically)
x_start = x0 + launch_inset_mm
ys = np.linspace(y0 + y_inset_mm, y1 - y_inset_mm, n_rays)
if start_jitter_mm > 0:
    ys = np.array([float(y + rng.uniform(-start_jitter_mm, start_jitter_mm)) for y in ys])

for y_start in ys:
    # skip seeds that start inside a planet
    if hits_planet(x_start, y_start):
        continue

    x, y = x_start, y_start
    vx, vy = v0_mm, 0.0                 # initial velocity to the right
    ax, ay = gravity_accel(x, y)
    pts = [(x, y)]

    for step in range(max_steps):
        v_target = v0_mm + (cruise_mm - v0_mm) * min(1.0, step / float(ramp_steps))
        ds  = step_mm
        dsm = ds / float(substeps)
        broke = False

        for _ in range(substeps):
            dt = dsm / max(1e-9, v_target)

            # half-kick
            vx += 0.5 * ax * dt
            vy += 0.5 * ay * dt
            # enforce target speed
            vm = math.hypot(vx, vy) or 1.0
            vx *= (v_target / vm)
            vy *= (v_target / vm)
            # drift
            x += vx * dt
            y += vy * dt

            if (not inside_rect(x, y)) or hits_planet(x, y):
                broke = True
                break

            # update accel + half-kick
            ax, ay = gravity_accel(x, y)
            vx += 0.5 * ax * dt
            vy += 0.5 * ay * dt

        if broke:
            break
        pts.append((x, y))

    if len(pts) >= 2:
        poly = Polyline(
            pts=pts,
            z_mode="midpoints",
            pos_offset=pos_off,
            settle_s=settle_s,
            min_seg_mm=0.6,
            max_seg_mm=1e9,
            feed_draw=feed_draw,
        )
        setattr(poly, "_plot_color", "#000000")
        pat.add(poly)

# 8) Preview & (optionally) execute
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat,
          step=50.0, figsize=(9.6, 6.2),
          title=f"Gravity scanlines: {n_rays} horizontal rays from left")

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0.0, 0.0)


### Pruned vector fields

In [ ]:
# --- Rays with hierarchical thinning by powers of two (round1: keep %2, round2: keep %4, round3: keep %8, …) ---
# Prereqs: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import math, random, numpy as np, penplot_widgets as ppw
from typing import List, Tuple
from pattern import Pattern, Polyline, Renderer, plot_plan

# ===== 1) Calibrated rectangle & Z map =====
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
corn = state["corners"]
grbl.set_surface_map(float(corn["BL"]["h"]), float(corn["BR"]["h"]),
                     float(corn["TL"]["h"]), float(corn["TR"]["h"]))

# ===== 2) Parameters =====
W, H = (x1 - x0), (y1 - y0)
rng_seed      = 1113
rng           = random.Random(rng_seed)

# planets for aesthetic bending
n_planets          = 12
r_min_frac, r_max_frac = 0.010, 0.028
circle_samples     = 80
soft_mm            = 3.0
G_eff              = 2.4
accel_clip         = 0.1
hit_margin_mm      = 0.6

# fan
origin_corner  = "BL"  # "BL","BR","TL","TR"
fan_center_deg = math.degrees(math.atan2((y0+H*0.5)-(y0 if origin_corner[0]=='B' else y1),
                                         (x0+W*0.5)-(x0 if origin_corner[1]=='L' else x1)))
fan_span_deg   = 120.0
n_rays         = 1*480  # dense; thinning will reduce locally

# integrator (constant-arclength-ish)
step_mm     = 0.6
substeps    = 3
v0_mm       = 1.0
cruise_mm   = 1.0
ramp_steps  = 40
max_steps   = 300
launch_inset_mm = 2.0

# thinning
chunk_len_mm      = 5.0        # compare in ~2 cm chunks
resample_step_mm  = 1.2         # distance sampling along rays
proximity_mm      = 1.5         # too-close threshold
neighbor_span     = 1           # compare i to i±1..i±neighbor_span
rounds_pow2       = 4           # 1→keep%2, 2→keep%4, 3→keep%8 (increase if you want more stages)

# motion / pen
pos_off   = -0.16
settle_s  = 0.06
feed_draw = 9000
execute   = False

# ===== 3) Scene (planets) =====
def circle_poly(cx, cy, r, n=circle_samples):
    return [(cx + r*math.cos(2*math.pi*i/n), cy + r*math.sin(2*math.pi*i/n)) for i in range(n+1)]

R0 = min(W, H)
planets = []
tries = 0
while len(planets) < n_planets and tries < 5000:
    tries += 1
    r  = rng.uniform(r_min_frac*R0, r_max_frac*R0)
    cx = rng.uniform(x0 + r + 2, x1 - r - 2)
    cy = rng.uniform(y0 + r + 2, y1 - r - 2)
    if all(math.hypot(cx-px, cy-py) >= 0.55*(r+pr) for (px,py,pr,_) in planets):
        planets.append((cx, cy, r, r*r))  # mass ∝ area (aesthetics)

def gravity_accel(x, y):
    ax = ay = 0.0
    for (cx, cy, r, m) in planets:
        dx, dy = (cx - x), (cy - y)
        d2 = dx*dx + dy*dy + soft_mm*soft_mm
        inv_d = 1.0 / math.sqrt(d2)
        inv_d3 = inv_d * inv_d * inv_d
        a = G_eff * m * inv_d3
        ax += a * dx; ay += a * dy
    mag = math.hypot(ax, ay)
    if mag > accel_clip:
        ax *= accel_clip / mag; ay *= accel_clip / mag
    return ax, ay

def inside_rect(x, y):
    return (x0 <= x <= x1) and (y0 <= y <= y1)

def hits_planet(x, y):
    for (cx, cy, r, _) in planets:
        if math.hypot(x - cx, y - cy) <= (r + hit_margin_mm):
            return True
    return False

def corner_xy(tag):
    if tag == "BL": return (x0+launch_inset_mm, y0+launch_inset_mm)
    if tag == "BR": return (x1-launch_inset_mm, y0+launch_inset_mm)
    if tag == "TL": return (x0+launch_inset_mm, y1-launch_inset_mm)
    return (x1-launch_inset_mm, y1-launch_inset_mm)

# ===== 4) Generate rays =====
ox, oy = corner_xy(origin_corner)
theta0 = math.radians(fan_center_deg)
thetas = np.linspace(theta0 - math.radians(fan_span_deg)*0.5,
                     theta0 + math.radians(fan_span_deg)*0.5, n_rays)

rays_pts: List[List[Tuple[float,float]]] = []
for th in thetas:
    x, y = ox, oy
    vx, vy = v0_mm*math.cos(th), v0_mm*math.sin(th)
    ax, ay = gravity_accel(x, y)
    pts = [(x, y)]
    for step in range(max_steps):
        v_target = v0_mm + (cruise_mm - v0_mm) * min(1.0, step/float(ramp_steps))
        ds  = step_mm
        dsm = ds / float(substeps)
        broke = False
        for _ in range(substeps):
            dt = dsm / max(1e-9, v_target)
            # half-kick
            vx += 0.5 * ax * dt; vy += 0.5 * ay * dt
            # speed set
            vm = math.hypot(vx, vy) or 1.0
            vx *= (v_target / vm); vy *= (v_target / vm)
            # drift
            x += vx * dt; y += vy * dt
            if (not inside_rect(x, y)) or hits_planet(x, y):
                broke = True; break
            # half-kick
            ax, ay = gravity_accel(x, y)
            vx += 0.5 * ax * dt; vy += 0.5 * ay * dt
        if broke: break
        pts.append((x, y))
    if len(pts) >= 2:
        rays_pts.append(pts)

# ===== 5) Resample → chunk → HIERARCHICAL thinning =====
XY = Tuple[float,float]

def resample_polyline(pts: List[XY], step: float) -> List[XY]:
    if len(pts) < 2 or step <= 0: return pts[:]
    d = [0.0]
    for i in range(1, len(pts)):
        x0,y0 = pts[i-1]; x1,y1 = pts[i]
        d.append(d[-1] + math.hypot(x1-x0, y1-y0))
    L = d[-1]
    if L < 1e-6: return pts[:1]
    n = max(1, int(L/step))
    targets = [i*L/n for i in range(n+1)]
    out = []
    j = 1
    for t in targets:
        while j < len(d) and d[j] < t: j += 1
        if j >= len(d): out.append(pts[-1]); break
        t0,t1 = d[j-1], d[j]
        if t1 - t0 < 1e-9: out.append(pts[j]); continue
        a = (t - t0)/(t1 - t0)
        x0,y0 = pts[j-1]; x1,y1 = pts[j]
        out.append((x0 + a*(x1-x0), y0 + a*(y1-y0)))
    return out

def chunk_indices_by_len(resampled: List[XY], step: float, chunk_len: float) -> List[Tuple[int,int]]:
    if len(resampled) < 2: return []
    pts_per_chunk = max(2, int(round(chunk_len/step)))
    chunks = []
    i = 0
    N = len(resampled)
    while i + 1 < N:
        j = min(N-1, i + pts_per_chunk)
        if j - i >= 1:
            chunks.append((i, j))
        i = j
    return chunks

def min_dist_between_sequences(a: List[XY], b: List[XY], window: int = 2) -> float:
    if not a or not b: return float('inf')
    m, n = len(a), len(b)
    mn = min(m, n)
    best = float('inf')
    for i in range(mn):
        x,y = a[i]
        j0 = max(0, i - window)
        j1 = min(n, i + window + 1)
        for j in range(j0, j1):
            u,v = b[j]
            d = (x-u)*(x-u) + (y-v)*(y-v)
            if d < best: best = d
    return math.sqrt(best)

rs = [resample_polyline(pts, resample_step_mm) for pts in rays_pts]
chunks_per_ray = [chunk_indices_by_len(r, resample_step_mm, chunk_len_mm) for r in rs]
keep = [ [True]*len(chunks_per_ray[i]) for i in range(len(rs)) ]

# hierarchical rounds: r=1 keep %2; r=2 keep %4; r=3 keep %8; …
for rpow in range(1, rounds_pow2+1):
    base = 2**rpow
    anchors = [i for i in range(len(rs)) if (i % base) == 0]
    # ensure all anchor chunks are kept this round (they may have been kept anyway)
    # candidates: everyone else
    for i in range(len(rs)):
        r_i = rs[i]; ch_i = chunks_per_ray[i]
        if not ch_i: continue
        is_anchor = (i % base) == 0
        for k, (i0,i1) in enumerate(ch_i):
            if not keep[i][k]: 
                continue
            if is_anchor:
                # never drop in this round
                continue
            segA = r_i[i0:i1+1]
            # test against anchor rays, and also against lower-index candidates to avoid crowding within tier
            drop_current = False
            # 1) anchors dominate all non-anchors this round
            for j in anchors:
                if j == i: continue
                ch_j = chunks_per_ray[j]
                # local neighborhood in index space for speed
                if abs(j - i) > max(4, neighbor_span): 
                    continue
                for kk in (k-1, k, k+1):
                    if 0 <= kk < len(ch_j) and keep[j][kk]:
                        j0,j1 = ch_j[kk]
                        segB = rs[j][j0:j1+1]
                        if min_dist_between_sequences(segA, segB, window=2) < proximity_mm:
                            keep[i][k] = False
                            drop_current = True
                            break
                if drop_current: break
            if drop_current: 
                continue
            # 2) among non-anchors in this round, keep the lower index; prune the higher if too close
            for dj in range(1, neighbor_span+1):
                j = i + dj
                if j >= len(rs): break
                if (j % base) == 0:  # anchor checked above
                    continue
                ch_j = chunks_per_ray[j]
                if k < len(ch_j) and keep[j][k]:
                    j0,j1 = ch_j[k]
                    segB = rs[j][j0:j1+1]
                    if min_dist_between_sequences(segA, segB, window=2) < proximity_mm:
                        # higher index loses
                        keep[j][k] = False

# reconstruct polylines from kept chunks (merge adjacent kept chunks)
thinned_pts: List[List[XY]] = []
for i, r in enumerate(rs):
    ch = chunks_per_ray[i]
    km = keep[i]
    if not ch: 
        continue
    # if *all* chunks are kept, just add the full ray (no breaks)
    if all(km):
        if len(r) >= 2:
            thinned_pts.append(r[:])
        continue
    # otherwise merge contiguous kept chunks
    runs = []
    cur = None
    for (k,(a,b)) in enumerate(ch):
        if km[k]:
            if cur is None: cur = [a,b]
            else:
                if a <= cur[1] + 1: cur[1] = max(cur[1], b)
                else: runs.append(tuple(cur)); cur = [a,b]
        else:
            if cur is not None: runs.append(tuple(cur)); cur = None
    if cur is not None: runs.append(tuple(cur))
    for (a,b) in runs:
        if b - a >= 1:
            thinned_pts.append(r[a:b+1])

# ===== 6) Build pattern (planets + rays) =====
pat = Pattern()
# planet outlines
for (cx, cy, r, _) in planets:
    poly = Polyline(
        pts=circle_poly(cx, cy, r),
        z_mode="midpoints",
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.6,
        max_seg_mm=1e9,
        feed_draw=3000,
    )
    setattr(poly, "_plot_color", "#000000")
    pat.add(poly)

# rays
for pts in thinned_pts:
    poly = Polyline(
        pts=pts,
        z_mode="midpoints",
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.6,
        max_seg_mm=1e9,
        feed_draw=feed_draw,
    )
    setattr(poly, "_plot_color", "#000000")
    pat.add(poly)

# ===== 7) Preview & (optionally) execute =====
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat,
          step=50.0, figsize=(9.6, 6.2),
          title=f"Hierarchical thinning (rounds={rounds_pow2}, proximity={proximity_mm} mm)")

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0.0, 0.0)


### Arrows planes

In [ ]:
# --- Gravity-fan + “arrow hooks” that branch RIGHT, then drop perpendicular back to the path, then return to tip (no pen lifts) ---
# Prereqs in notebook: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import math, random, numpy as np, penplot_widgets as ppw
from typing import List, Tuple, Optional
from pattern import Pattern, Polyline, Renderer, plot_plan

XY = Tuple[float, float]

# 1) Load calibrated rectangle (+ program Z surface map for consistent Z)
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
corn = state["corners"]
grbl.set_surface_map(float(corn["BL"]["h"]), float(corn["BR"]["h"]),
                     float(corn["TL"]["h"]), float(corn["TR"]["h"]))

# 2) Parameters (close to your tuned values) -------------------------------------
W, H = (x1 - x0), (y1 - y0)
rng_seed           = 1113
n_planets          = 20
r_min_frac, r_max_frac = 0.008, 0.05  # radius as fraction of min(W,H)
min_sep_frac       = 0.05
circle_samples     = 90

origin_corner      = "BL"            # "BL","BR","TL","TR"
fan_center_deg     = math.degrees(math.atan2((y0+H*0.5)-(y0 if origin_corner[0]=='B' else y1),
                                             (x0+W*0.5)-(x0 if origin_corner[1]=='L' else x1)))
fan_span_deg       = 120.0
n_rays             = 80

# Integration / motion (smooth arclength + speed ramp)
step_mm     = 0.5     # arclength per EMITTED vertex (smaller = smoother)
substeps    = 4       # micro-steps per emitted vertex
v0_mm       = 5.0     # starting speed magnitude
cruise_mm   = 5.0     # target speed after ramp
ramp_steps  = 60      # vertices to ramp v0 → cruise
max_steps   = 620     # cap emitted vertices per ray

soft_mm            = 1.0   # gravitational softening
G_eff              = 15.0  # overall gravity strength
accel_clip         = 1.0   # clamp |a| to keep curvature sane
hit_margin_mm      = -0.5  # stop if inside (r + margin)
launch_inset_mm    = 0.0   # start slightly inside the rectangle

# Arrow-hook parameters (INLINE, no pen lift)
hook_every_mm      = 10.0   # spacing along baseline between hooks (tip-to-tip)
hook_back_mm       = 2.0    # how far to go “backwards” from the tip along a 30° right slant
hook_angle_deg     = 30.0   # the “backwards” slant angle (relative to the reverse of the local tangent)
hook_start_skip_mm = 50.0   # skip first part (near origin) before first hook
use_right_side_only = True  # only shoot the perpendicular on the right side; if no hit, skip hook

# Motion / pen
pos_off   = -0.20
settle_s  = 0.25
feed_draw = 6000
execute   = False

rng = random.Random(rng_seed)

# 3) Helpers --------------------------------------------------------------------
def corner_xy(tag: str) -> XY:
    if tag == "BL": return (x0+launch_inset_mm, y0+launch_inset_mm)
    if tag == "BR": return (x1-launch_inset_mm, y0+launch_inset_mm)
    if tag == "TL": return (x0+launch_inset_mm, y1-launch_inset_mm)
    return (x1-launch_inset_mm, y1-launch_inset_mm)

def circle_poly(cx, cy, r, n=circle_samples):
    return [(cx + r*math.cos(2*math.pi*i/n), cy + r*math.sin(2*math.pi*i/n)) for i in range(n+1)]

def inside_rect(x, y):
    return (x0 <= x <= x1) and (y0 <= y <= y1)

# Polyline arc-length utilities
def cumulative_lengths(pts: List[XY]) -> List[float]:
    L = [0.0]
    for i in range(1, len(pts)):
        dx = pts[i][0]-pts[i-1][0]; dy = pts[i][1]-pts[i-1][1]
        L.append(L[-1] + math.hypot(dx, dy))
    return L

def interp_at_s(pts: List[XY], cum: List[float], s: float) -> Tuple[float,float,int,float]:
    """Return (x,y,idx,t) where idx is segment start index and t∈[0,1] within that segment."""
    if s <= 0.0: return pts[0][0], pts[0][1], 0, 0.0
    if s >= cum[-1]: return pts[-1][0], pts[-1][1], len(pts)-2, 1.0
    # binary search
    lo, hi = 0, len(cum)-1
    while lo+1 < hi:
        mid = (lo+hi)//2
        if cum[mid] <= s: lo = mid
        else: hi = mid
    seg_len = max(1e-12, cum[lo+1]-cum[lo])
    t = (s - cum[lo]) / seg_len
    x = pts[lo][0] + t*(pts[lo+1][0]-pts[lo][0])
    y = pts[lo][1] + t*(pts[lo+1][1]-pts[lo][1])
    return x, y, lo, t

# 2D vector helpers
def rot_ccw(vx: float, vy: float, ang_rad: float) -> XY:
    c, s = math.cos(ang_rad), math.sin(ang_rad)
    return (c*vx - s*vy, s*vx + c*vy)

def cross(ax, ay, bx, by):
    return ax*by - ay*bx

def line_hits_polyline_from_point_dir(A: XY, dir_vec: XY, poly: List[XY], max_idx: int) -> Optional[XY]:
    """
    Intersect the infinite line L(s)=A+s*dir with the polyline segments poly[k]->poly[k+1] for k∈[0,max_idx].
    Returns the closest intersection point *in the +dir direction* (s>0).
    """
    ax, ay = A; dx, dy = dir_vec
    best_s = None; best_pt = None
    for k in range(0, max_idx+1):
        x1,y1 = poly[k]; x2,y2 = poly[k+1]
        sx, sy = (x2-x1, y2-y1)
        denom = cross(sx, sy, dx, dy)
        if abs(denom) < 1e-12:
            continue  # parallel
        # Solve: A + s*dir = S1 + t*seg  →  cross((S1 - A), dir)/cross(seg, dir) = t
        t = cross((ax - x1), (ay - y1), dx, dy) / denom
        s = cross((ax - x1), (ay - y1), sx, sy) / denom
        if 0.0 <= t <= 1.0 and s > 1e-6:
            ix, iy = (x1 + t*sx, y1 + t*sy)
            if best_s is None or s < best_s:
                best_s = s; best_pt = (ix, iy)
    return best_pt

def insert_hooks_arrow(pts: List[XY],
                       spacing: float,
                       back_len: float,
                       angle_deg: float,
                       start_skip: float,
                       right_only: bool = True) -> List[XY]:
    """
    “Arrow hook” at every spacing (after start_skip):
      1) Draw baseline up to the hook tip P (on the path).
      2) From P, go BACKWARDS by `back_len` at +30° toward the RIGHT relative to the reverse-tangent.
      3) From that point A, drop a perpendicular to the baseline (search intersection in +right normal first;
         if right_only=False, try both directions).
      4) From the intersection I, draw back to P (retrace allowed). Continue baseline.
    """
    if len(pts) < 2:
        return pts[:]

    cum = cumulative_lengths(pts)
    total = cum[-1]
    if total < start_skip + spacing:
        return pts[:]

    out: List[XY] = [pts[0]]
    s_next = start_skip
    i_seg = 0
    ang_back = math.radians(angle_deg)  # 30°

    def append_until_s(s_target: float):
        nonlocal i_seg
        while i_seg < len(pts)-1 and cum[i_seg+1] <= s_target + 1e-9:
            out.append(pts[i_seg+1])
            i_seg += 1
        if s_target <= cum[-1] - 1e-9:
            x,y,_,_ = interp_at_s(pts, cum, s_target)
            if not out or (abs(out[-1][0]-x) > 1e-9 or abs(out[-1][1]-y) > 1e-9):
                out.append((x,y))

    while s_next <= total - 1e-9:
        # Tip P
        append_until_s(s_next)
        xP, yP, idx, _t = interp_at_s(pts, cum, s_next)
        if idx >= len(pts)-1: break

        # Local tangent at idx (forward direction along path)
        tx = pts[idx+1][0] - pts[idx][0]
        ty = pts[idx+1][1] - pts[idx][1]
        nrm = math.hypot(tx, ty)
        if nrm < 1e-9:
            s_next += spacing
            continue
        tx /= nrm; ty /= nrm

        # Right-hand normal (for perpendicular search)
        nx, ny = (ty, -tx)

        # “Backwards 30° to the RIGHT”: rotate t by +150° (CCW) → unit vector
        bx, by = rot_ccw(tx, ty, math.radians(150.0))
        A = (xP + back_len*bx, yP + back_len*by)

        # From A, drop a perpendicular toward RIGHT first (direction = +n)
        I = None
        dir_try = [(nx, ny)]
        if not right_only:
            dir_try.append((-nx, -ny))

        for dtry in dir_try:
            hit = line_hits_polyline_from_point_dir(A, dtry, pts, max_idx=idx)
            if hit is not None:
                I = hit
                break

        # If no intersection on requested side(s), skip this hook
        if I is None:
            s_next += spacing
            continue

        # Clip triangle vertices inside drawing rect
        if not (inside_rect(A[0], A[1]) and inside_rect(I[0], I[1])):
            s_next += spacing
            continue

        # Ensure we're exactly at tip P in output (after append_until_s)
        if not (abs(out[-1][0]-xP) <= 1e-9 and abs(out[-1][1]-yP) <= 1e-9):
            out.append((xP, yP))

        # Draw: P -> A -> I -> P (returns to tip)
        out.append((A[0], A[1]))
        out.append((I[0], I[1]))
        out.append((xP, yP))

        s_next += spacing

    # Tail (rest of baseline)
    append_until_s(total)
    return out

# 4) Scatter planets ------------------------------------------------------------
R0 = min(W, H)
min_sep = min_sep_frac * R0
planets = []  # (cx,cy,r,mass)
tries = 0
while len(planets) < n_planets and tries < 5000:
    tries += 1
    r = rng.uniform(r_min_frac*R0, r_max_frac*R0)
    cx = rng.uniform(x0 + r + 2.0, x1 - r - 2.0)
    cy = rng.uniform(y0 + r + 2.0, y1 - r - 2.0)
    ok = True
    for (px, py, pr, _) in planets:
        if math.hypot(cx - px, cy - py) < max(min_sep, 0.65*(r + pr)):
            ok = False; break
    if ok:
        m = (r*r)  # mass ∝ area
        planets.append((cx, cy, r, m))

# 5) Planet outlines (optional)
pat = Pattern()
for (cx, cy, r, _) in planets:
    poly = Polyline(
        pts=circle_poly(cx, cy, r),
        z_mode="threshold",
        feed_draw=600,
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.5,
        max_seg_mm=1e9,
    )
    setattr(poly, "_plot_color", "#000000")
    pat.add(poly)

# 6) Gravity + integrator -------------------------------------------------------
def gravity_accel(x, y):
    ax = ay = 0.0
    for (cx, cy, r, m) in planets:
        dx, dy = (cx - x), (cy - y)
        d2 = dx*dx + dy*dy + soft_mm*soft_mm
        inv_d = 1.0 / math.sqrt(d2)
        inv_d3 = inv_d * inv_d * inv_d
        a = G_eff * m * inv_d3
        ax += a * dx
        ay += a * dy
    mag = math.hypot(ax, ay)
    if mag > accel_clip:
        ax *= accel_clip / mag
        ay *= accel_clip / mag
    return ax, ay

def hits_planet(x, y):
    for (cx, cy, r, _) in planets:
        if math.hypot(x - cx, y - cy) <= (r + hit_margin_mm):
            return True
    return False

# 7) Launch fan (velocity-Verlet + constant arclength; then add hooks) ----------
ox, oy = corner_xy(origin_corner)
theta0 = math.radians(fan_center_deg)
thetas = np.linspace(theta0 - math.radians(fan_span_deg)*0.5,
                     theta0 + math.radians(fan_span_deg)*0.5, n_rays)

for th in thetas:
    x, y = ox, oy
    vx, vy = v0_mm*math.cos(th), v0_mm*math.sin(th)
    ax, ay = gravity_accel(x, y)
    pts = [(x, y)]

    for step in range(max_steps):
        # ramp speed target
        v_target = v0_mm + (cruise_mm - v0_mm) * min(1.0, step/float(ramp_steps))
        ds  = step_mm
        dsm = ds / float(substeps)
        broke = False

        for _ in range(substeps):
            dt = dsm / max(1e-9, v_target)

            # half-kick
            vx += 0.5 * ax * dt
            vy += 0.5 * ay * dt
            # renorm to target speed
            vm = math.hypot(vx, vy) or 1.0
            vx *= (v_target / vm)
            vy *= (v_target / vm)
            # drift
            x += vx * dt
            y += vy * dt
            if (not inside_rect(x, y)) or hits_planet(x, y):
                broke = True
                break
            # accel at new pos + half-kick
            ax, ay = gravity_accel(x, y)
            vx += 0.5 * ax * dt
            vy += 0.5 * ay * dt

        if broke:
            break
        pts.append((x, y))

    # Insert “arrow hooks” inline, exactly as you described
    pts_hooked = insert_hooks_arrow(
        pts=pts,
        spacing=hook_every_mm,
        back_len=hook_back_mm,
        angle_deg=hook_angle_deg,
        start_skip=hook_start_skip_mm,
        right_only=use_right_side_only
    )

    if len(pts_hooked) >= 2:
        poly = Polyline(
            pts=pts_hooked,
            z_mode="midpoints",       # smooth & fast
            pos_offset=pos_off,
            settle_s=settle_s,
            min_seg_mm=0.6,
            max_seg_mm=1e9,
            feed_draw=feed_draw,
        )
        setattr(poly, "_plot_color", "#000000")
        pat.add(poly)

# 8) Preview & (optionally) execute --------------------------------------------
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat,
          step=50.0, figsize=(9.6, 6.2),
          title=f"Gravity fan with RIGHT-side arrow hooks (every {hook_every_mm} mm; back {hook_back_mm} mm at {hook_angle_deg}°)")

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0, 0)


### Arrow by time + friction + non-2.0 gravity

In [ ]:
# --- Gravity-fan with time-based RIGHT-side arrow hooks (optionally symmetric), friction, and configurable gravity exponent ---
# Prereqs: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import math, random, numpy as np, penplot_widgets as ppw
from typing import List, Tuple
from pattern import Pattern, Polyline, Renderer, plot_plan

XY = Tuple[float, float]

# 1) Load calibrated rectangle (+ program Z surface map)
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])
corn = state["corners"]
grbl.set_surface_map(float(corn["BL"]["h"]), float(corn["BR"]["h"]),
                     float(corn["TL"]["h"]), float(corn["TR"]["h"]))

# 2) Parameters -----------------------------------------------------------------
W, H = (x1 - x0), (y1 - y0)
rng_seed           = 1113
n_planets          = 14 #14
r_min_frac, r_max_frac = 0.02, 0.06    # radii as fraction of min(W,H)
min_sep_frac       = 0.05
circle_samples     = 90

origin_corner      = "BL"               # "BL","BR","TL","TR"
fan_center_deg     = math.degrees(math.atan2((y0+H*0.5)-(y0 if origin_corner[0]=='B' else y1),
                                             (x0+W*0.5)-(x0 if origin_corner[1]=='L' else x1)))
fan_span_deg       = 120.0
n_rays             = 150 #150

# --- Physics / integration (fixed dt in real time) -----------------------------
dt_phys      = 0.02       # seconds per microstep
substeps     = 2          # microsteps per emitted vertex
emit_every   = 1          # emit a vertex after this many microsteps
max_emits    = 5000       # cap emitted vertices per ray
v0_mm        = 15.0       # initial speed (mm/s)
gamma_drag   = 0.08       # linear friction coefficient (s^-1); larger = stronger slowdown
max_speed    = 120.0      # clamp to avoid huge leaps (mm/s)

# Gravity model: a ∝ 1/r^(alpha) along r̂, with softening
G_eff        = 10000.0
alpha        = 3.0        # 2.0 = Newtonian; try ~1.6 or ~2.6 for different looks
soft_mm      = 0.0
accel_clip   = 1_000_000.0
hit_margin_mm = -0.5
launch_inset_mm = 0.0

# --- Arrow hooks (RIGHT side, time-based) -------------------------------------
hook_start_skip_t  = 1.5   # seconds before first arrow
hook_every_t       = 0.30  # seconds between arrow tips
hook_back_mm       = 1.5   # back slant length from tip
hook_angle_deg     = 30.0  # slant angle (relative to REVERSE tangent)
clip_hooks_to_rect = True  # ignore hooks with vertices out of bounds

# NEW: make symmetric arrow heads by going past the line to the mirrored point
symmetric_arrows   = True  # set False to revert to right-side-only “drop to line” hooks

# Motion / pen
pos_off   = -0.20
settle_s  = 0.25
feed_draw = 10000
execute   = False

rng = random.Random(rng_seed)

# 3) Helpers --------------------------------------------------------------------
def corner_xy(tag: str) -> XY:
    if tag == "BL": return (x0+launch_inset_mm, y0+launch_inset_mm)
    if tag == "BR": return (x1-launch_inset_mm, y0+launch_inset_mm)
    if tag == "TL": return (x0+launch_inset_mm, y1-launch_inset_mm)
    return (x1-launch_inset_mm, y1-launch_inset_mm)

def circle_poly(cx, cy, r, n=circle_samples):
    return [(cx + r*math.cos(2*math.pi*i/n), cy + r*math.sin(2*math.pi*i/n)) for i in range(n+1)]

def inside_rect(x, y):
    return (x0 <= x <= x1) and (y0 <= y <= y1)

def rot_ccw(vx: float, vy: float, ang_rad: float) -> XY:
    c, s = math.cos(ang_rad), math.sin(ang_rad)
    return (c*vx - s*vy, s*vx + c*vy)

# 4) Planets --------------------------------------------------------------------
R0 = min(W, H)
min_sep = min_sep_frac * R0
planets = []  # (cx,cy,r,mass)
tries = 0
while len(planets) < n_planets and tries < 5000:
    tries += 1
    r = rng.uniform(r_min_frac*R0, r_max_frac*R0)
    cx = rng.uniform(x0 + r + 2.0, x1 - r - 2.0)
    cy = rng.uniform(y0 + r + 2.0, y1 - r - 2.0)
    ok = True
    for (px, py, pr, _) in planets:
        if math.hypot(cx - px, cy - py) < max(min_sep, 0.65*(r + pr)):
            ok = False; break
    if ok:
        m = (r*r)  # mass ∝ area
        planets.append((cx, cy, r, m))

# 5) Optional planet outlines
pat = Pattern()
for (cx, cy, r, _) in planets:
    poly = Polyline(
        pts=circle_poly(cx, cy, r),
        z_mode="threshold",
        feed_draw=600,
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.5,
        max_seg_mm=1e9,
    )
    setattr(poly, "_plot_color", "#000000")
    pat.add(poly)

# 6) Physics: gravity with exponent + friction (fixed dt) -----------------------
def gravity_accel(x, y):
    """a = Σ G*m * r̂ / r^alpha  →  G*m*(dx,dy)/r^(alpha+1), with softening."""
    ax = ay = 0.0
    for (cx, cy, r, m) in planets:
        dx, dy = (cx - x), (cy - y)
        d2s = dx*dx + dy*dy + (soft_mm*soft_mm if soft_mm > 0 else 0.0)
        if d2s <= 1e-12:
            continue
        inv_r_pow = d2s**(-0.5*(alpha + 1.0))   # 1 / r^(alpha+1)
        a_scale = G_eff * m * inv_r_pow
        ax += a_scale * dx
        ay += a_scale * dy
    amag = math.hypot(ax, ay)
    if amag > accel_clip:
        ax *= accel_clip/amag
        ay *= accel_clip/amag
    return ax, ay

def integrate_ray(ox, oy, theta, t_max_emit: int) -> Tuple[List[XY], List[float]]:
    """
    Semi-implicit Euler with linear drag (fixed real-time dt):
      v ← v + a*dt;  v ← v*(1 - γ*dt);  clamp |v|;  x ← x + v*dt.
    Emit one vertex every `emit_every` microsteps. Return (points, times).
    """
    x, y = ox, oy
    vx, vy = v0_mm*math.cos(theta), v0_mm*math.sin(theta)
    pts: List[XY] = [(x, y)]
    ts:  List[float] = [0.0]
    t = 0.0
    emits = 1

    while emits < t_max_emit:
        broke = False
        for _ in range(substeps):
            ax, ay = gravity_accel(x, y)
            # velocity + drag
            vx += ax * dt_phys
            vy += ay * dt_phys
            drag = max(0.0, 1.0 - gamma_drag*dt_phys)
            vx *= drag; vy *= drag
            # clamp |v|
            vmag = math.hypot(vx, vy)
            if vmag > max_speed:
                vx *= max_speed/vmag; vy *= max_speed/vmag
            # drift
            x += vx * dt_phys
            y += vy * dt_phys
            t += dt_phys

            # stop when leaving rect or hitting a planet
            if (x < x0 or x > x1 or y < y0 or y > y1) or \
               any(math.hypot(x - cx, y - cy) <= (r + hit_margin_mm) for (cx,cy,r,_) in planets):
                broke = True
                break

        if broke:
            break

        if emits % emit_every == 0:
            pts.append((x, y))
            ts.append(t)
        emits += 1

    return pts, ts

# 7) RIGHT-side hook insertion at *precise times* (with symmetric option) -------
def insert_time_hooks_right(pts: List[XY],
                            ts: List[float],
                            every_t: float,
                            start_skip_t: float,
                            back_len: float,
                            back_angle_deg: float,
                            symmetric: bool = False) -> List[XY]:
    """
    For tips at t = start_skip_t + k*every_t:
      Tip P via linear time interpolation → local tangent t̂ from segment i→i+1.
      Reverse tangent r̂ = -t̂; rotate by +angle to RIGHT → b̂; A = P + back_len*b̂.
      If symmetric=False: project A onto tangent line at P (I), append P→A→I→P.
      If symmetric=True: mirror A across tangent line at P to A_sym and append P→A→A_sym→P.
    """
    if len(pts) < 3 or ts[-1] < start_skip_t + every_t:
        return pts[:]

    def interp_t_with_idx(tq: float) -> Tuple[float, float, int, float]:
        if tq <= ts[0]: return pts[0][0], pts[0][1], 0, 0.0
        if tq >= ts[-1]: return pts[-1][0], pts[-1][1], len(pts)-2, 1.0
        i = max(0, int(np.searchsorted(ts, tq) - 1))
        j = min(len(ts)-1, i+1)
        dT = max(1e-12, ts[j] - ts[i])
        a = (tq - ts[i]) / dT
        x = pts[i][0] + a*(pts[j][0]-pts[i][0])
        y = pts[i][1] + a*(pts[j][1]-pts[i][1])
        return x, y, i, a

    out: List[XY] = []
    out.extend(pts)  # keep baseline
    inserts: List[Tuple[int, List[XY]]] = []
    t_tip = start_skip_t
    ang_back = math.radians(back_angle_deg)
    t_last_safe = ts[-2] if len(ts) >= 2 else ts[-1]

    while t_tip <= t_last_safe + 1e-9:
        xP, yP, i, a = interp_t_with_idx(t_tip)
        sx, sy = (pts[i+1][0] - pts[i][0], pts[i+1][1] - pts[i][1])
        n = math.hypot(sx, sy)
        if n < 1e-9:
            t_tip += every_t; continue
        tx, ty = sx/n, sy/n

        # build off-tip point A to the RIGHT (backwards by 'back_len' at +angle)
        rx, ry = (-tx, -ty)
        bx, by = rot_ccw(rx, ry, +ang_back)
        Ax, Ay = (xP + back_len*bx, yP + back_len*by)

        APx, APy = (Ax - xP, Ay - yP)
        dot_AP_t = APx*tx + APy*ty

        if not symmetric:
            # project A back to tangent line: I = P + proj(AP, t̂)
            Ix, Iy = (xP + dot_AP_t*tx, yP + dot_AP_t*ty)
            if clip_hooks_to_rect and (not inside_rect(Ax,Ay) or not inside_rect(Ix,Iy)):
                t_tip += every_t; continue
            hook = [(xP, yP), (Ax, Ay), (Ix, Iy), (xP, yP)]
        else:
            # symmetric across tangent: A_sym = P + 2*proj(AP,t̂) - AP
            A_symx = xP + 2*dot_AP_t*tx - APx
            A_symy = yP + 2*dot_AP_t*ty - APy
            if clip_hooks_to_rect and (not inside_rect(Ax,Ay) or not inside_rect(A_symx,A_symy)):
                t_tip += every_t; continue
            hook = [(xP, yP), (Ax, Ay), (A_symx, A_symy), (xP, yP)]

        ins_after = min(len(out)-1, max(1, i))
        inserts.append((ins_after, hook))
        t_tip += every_t

    inserts.sort(key=lambda z: z[0])
    merged: List[XY] = []
    j = 0
    for i in range(len(out)):
        merged.append(out[i])
        while j < len(inserts) and inserts[j][0] == i:
            merged.extend(inserts[j][1])
            j += 1
    return merged

# 8) Launch fan, integrate, add hooks, add to Pattern ---------------------------
ox, oy = corner_xy(origin_corner)
theta0 = math.radians(fan_center_deg)
thetas = np.linspace(theta0 - math.radians(fan_span_deg)*0.5,
                     theta0 + math.radians(fan_span_deg)*0.5, n_rays)

for th in thetas:
    pts, ts = integrate_ray(ox, oy, th, t_max_emit=max_emits)
    if len(pts) < 2:
        continue

    pts_with_hooks = insert_time_hooks_right(
        pts=pts,
        ts=ts,
        every_t=hook_every_t,
        start_skip_t=hook_start_skip_t,
        back_len=hook_back_mm,
        back_angle_deg=hook_angle_deg,
        symmetric=symmetric_arrows,
    )

    if len(pts_with_hooks) >= 2:
        poly = Polyline(
            pts=pts_with_hooks,
            z_mode="threshold",
            z_threshold=0.5,
            pos_offset=pos_off,
            settle_s=settle_s,
            min_seg_mm=0.6,
            max_seg_mm=1e9,
            feed_draw=feed_draw,
        )
        setattr(poly, "_plot_color", "#000000")
        pat.add(poly)

# 9) Preview & (optionally) execute --------------------------------------------
title = f"Gravity fan (α={alpha}, drag={gamma_drag}, hooks Δt={hook_every_t}s, back={hook_back_mm}mm, symmetric={symmetric_arrows})"
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat,
          step=50.0, figsize=(9.6, 6.2), title=title)

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0, 0)


### Sauqares

In [ ]:
# --- Grid of squircles with quadratic fall, sequential occlusion, then recenter ---
# Prereqs: cfg, grbl, Pattern/Polyline/Renderer/plot_plan, and a saved rectangle via the widget.

import math, random, penplot_widgets as ppw
from typing import List, Tuple
from pattern import Pattern, Polyline, Renderer, plot_plan

XY = Tuple[float, float]

# 1) Load drawing rectangle (from the calibration widget)
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])

# 2) Parameters
n_cols         = 11         # squircles per row
n_rows         = 14         # rows
side_mm        = 7.0        # nominal side length
gutter_x_mm    = 0.0        # spacing x
gutter_y_mm    = 0.0        # spacing y
margin_mm      = 0.0

rot_step_deg   = 4.0        # jitter grows by row index: ±(row*rot_step)
off_step_mm    = 0.4        # position jitter grows by row index: ±(row*off_step)

# Per-row vertical fall. Absolute displacement grows quadratically with row index.
fall_v0_mm     = 0.0        # base per-row downward shift
fall_v_step_mm = 0.8        # increment per row

# Squircleness control
# p = 2 gives a circle, larger p approaches a rounded square. Try 3 to 8.
squircleness_p = 4.0
samples_per_shape = 80      # polygon sampling of the superellipse

rng_seed       = 2026
pos_off        = -0.15
settle_s       = 0.15
feed_draw      = 1500
execute        = False

rng = random.Random(rng_seed)

# 3) Layout
W, H = (x1 - x0), (y1 - y0)
row_h = side_mm + gutter_y_mm
col_w = side_mm + gutter_x_mm

total_w = n_cols*side_mm + (n_cols-1)*gutter_x_mm
total_h = n_rows*side_mm + (n_rows-1)*gutter_y_mm
assert total_w + 2*margin_mm <= W + 1e-6, "Increase rect or reduce width or margins."
assert total_h + 2*margin_mm <= H + 1e-6, "Increase rect or reduce height or margins."

x_start = x0 + 0.5*(W - total_w)
y_start = y0 + 0.5*(H - total_h)

def squirkle_outline(center_x: float, center_y: float, side: float,
                     rot_deg: float = 0.0, p: float = 4.0, samples: int = 80) -> List[XY]:
    """
    Superellipse based squirkle centered at (cx, cy).
    |x/a|^p + |y/b|^p = 1 with a = b = side/2.
    p = 2 gives a circle, p -> inf approaches a square with rounded corners.
    Returns a closed list of points.
    """
    cx, cy = center_x, center_y
    a = b = side * 0.5
    th = math.radians(rot_deg)
    c, s = math.cos(th), math.sin(th)

    pts: List[XY] = []
    for i in range(samples):
        t = 2.0 * math.pi * i / samples
        ct, st = math.cos(t), math.sin(t)
        # superellipse parametric form
        x = a * (abs(ct) ** (2.0 / p)) * (1 if ct >= 0 else -1)
        y = b * (abs(st) ** (2.0 / p)) * (1 if st >= 0 else -1)
        # rotate
        xr = c * x - s * y
        yr = s * x + c * y
        pts.append((cx + xr, cy + yr))
    # close the loop
    if pts[0] != pts[-1]:
        pts.append(pts[0])
    return pts

# ---------- Convex clipping helpers (occlude older lines under a newer shape) ----------
def _cross(ax, ay, bx, by): return ax*by - ay*bx

def point_in_convex_polygon(p: XY, poly: List[XY]) -> bool:
    x, y = p
    n = len(poly)-1 if poly[0] == poly[-1] else len(poly)
    if n < 3: return False
    sign = 0
    for i in range(n):
        x1,y1 = poly[i]; x2,y2 = poly[(i+1)%n]
        c = _cross(x2-x1, y2-y1, x-x1, y-y1)
        if c != 0:
            if sign == 0: sign = 1 if c > 0 else -1
            elif (c > 0 and sign < 0) or (c < 0 and sign > 0):
                return False
    return True

def segment_poly_intersections(A: XY, B: XY, poly: List[XY]) -> List[float]:
    x1,y1 = A; x2,y2 = B
    dx, dy = (x2-x1, y2-y1)
    ts = []
    m = len(poly)-1 if poly[0] == poly[-1] else len(poly)
    for i in range(m):
        sx, sy = poly[i]; ex, ey = poly[(i+1)%m]
        ux, uy = (ex - sx, ey - sy)
        denom = _cross(dx, dy, ux, uy)
        if abs(denom) < 1e-12:
            continue
        t = _cross(sx - x1, sy - y1, ux, uy) / denom
        u = _cross(sx - x1, sy - y1, dx, dy) / denom
        if 0.0 < t < 1.0 and 0.0 <= u <= 1.0:
            ts.append(t)
    ts.sort()
    return ts

def subtract_convex_polygon_from_polyline(pts: List[XY], poly: List[XY]) -> List[List[XY]]:
    if len(pts) < 2: return []
    out: List[List[XY]] = []
    cur: List[XY] = []

    for i in range(len(pts)-1):
        A = pts[i]; B = pts[i+1]
        tlist = [0.0] + segment_poly_intersections(A, B, poly) + [1.0]
        for k in range(len(tlist)-1):
            t0, t1 = tlist[k], tlist[k+1]
            if t1 - t0 < 1e-9: continue
            mid = (t0 + t1) * 0.5
            xm = A[0] + (B[0]-A[0])*mid
            ym = A[1] + (B[1]-A[1])*mid
            inside = point_in_convex_polygon((xm,ym), poly)
            if inside:
                if len(cur) >= 2:
                    out.append(cur); cur = []
                continue
            x0p = A[0] + (B[0]-A[0])*t0
            y0p = A[1] + (B[1]-A[1])*t0
            x1p = A[0] + (B[0]-A[0])*t1
            y1p = A[1] + (B[1]-A[1])*t1
            if not cur:
                cur = [(x0p,y0p), (x1p,y1p)]
            else:
                if math.hypot(cur[-1][0]-x0p, cur[-1][1]-y0p) > 1e-9:
                    cur.append((x0p,y0p))
                cur.append((x1p,y1p))
    if len(cur) >= 2:
        out.append(cur)
    return out

# --- Quadratic fall displacement (per row r, 0-indexed) ---
def fall_displacement_y(r: int) -> float:
    """
    Each row contributes an additional per-row velocity in Y:
      v_r = fall_v0_mm + r * fall_v_step_mm
    Absolute displacement:
      Δy(r) = (r+1)*fall_v0_mm + 0.5*fall_v_step_mm * r*(r+1)
    """
    return (r + 1) * fall_v0_mm + 0.5 * fall_v_step_mm * r * (r + 1)

# 4) Sequential build with occlusion: squirkle by squirkle within each row
strokes: List[List[XY]] = []  # older first; newest appended last

for r in range(n_rows):
    cy_base = y_start + r*row_h + side_mm*0.5
    cy = cy_base + fall_displacement_y(r)

    max_rot = r * rot_step_deg
    max_off = r * off_step_mm

    for c in range(n_cols):
        cx = x_start + c*col_w + side_mm*0.5

        rot = rng.uniform(-max_rot, +max_rot)
        dx  = rng.uniform(-max_off, +max_off)
        dy  = rng.uniform(-max_off, +max_off)

        poly_shape = squirkle_outline(
            cx + dx, cy + dy, side_mm,
            rot_deg=rot, p=squircleness_p, samples=samples_per_shape
        )  # closed loop

        # 4a) Cut existing strokes beneath this new squirkle
        if strokes:
            kept: List[List[XY]] = []
            for s in strokes:
                pieces = subtract_convex_polygon_from_polyline(s, poly_shape)
                kept.extend(pieces)
            strokes = kept

        # 4b) Add the outline of the new squirkle on top
        strokes.append(poly_shape)

# 5) Recenter the whole composition
if strokes:
    minx = min(p[0] for s in strokes for p in s)
    maxx = max(p[0] for s in strokes for p in s)
    miny = min(p[1] for s in strokes for p in s)
    maxy = max(p[1] for s in strokes for p in s)
    cx_cur = 0.5*(minx + maxx)
    cy_cur = 0.5*(miny + maxy)
    cx_tar = x0 + 0.5*W
    cy_tar = y0 + 0.5*H
    dx_c = cx_tar - cx_cur
    dy_c = cy_tar - cy_cur
    strokes = [[(x+dx_c, y+dy_c) for (x,y) in s] for s in strokes]

# 6) Convert to Pattern
pat = Pattern()
for pts in strokes:
    if len(pts) < 2: continue
    polyline = Polyline(
        pts=pts,
        z_mode="midpoints",
        pos_offset=pos_off,
        settle_s=settle_s,
        min_seg_mm=0.2,
        max_seg_mm=1e9,
        feed_draw=feed_draw,
    )
    setattr(polyline, "_plot_color", "#000000")
    pat.add(polyline)

# 7) Preview and optionally execute
title = (
    f"Squircles with quadratic fall (v0={fall_v0_mm} mm/row, dv={fall_v_step_mm} mm/row^2), "
    f"p={squircleness_p}, samples={samples_per_shape}, sequential occlusion, recentered"
)
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat,
          step=50.0, figsize=(9.2, 6.2), title=title)

if execute:
    Renderer(grbl).run(pat)
    # grbl.goto_abs(0, 0)


### next

In [ ]:
# Gaussian-jittered grid with center-weighted noise and circle size
# Grid lines are chained into long polylines for faster plotting.
# Two execution switches let you draw grid and circles with different pens.

import math, random, penplot_widgets as ppw
from typing import List, Tuple, Iterable
from pattern import Pattern, Polyline, Renderer, plot_plan

XY = Tuple[float, float]

# 1) Load drawing rectangle (from the calibration widget)
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])

# 2) Parameters
n_cols                = 17       # grid points in X
n_rows                = 12       # grid points in Y
margin_mm             = 0.0
grid_dx_mm            = 10.0     # spacing X
grid_dy_mm            = 10.0     # spacing Y

rng_seed              = 2025

# Jitter strongest at page center with Gaussian falloff
jitter_sigma_center   = 2.2      # mm, std dev at center
jitter_falloff_sigma  = 40.0     # mm, radial falloff sigma

# Circles at each grid point
circle_base_diam_mm   = 2.0
circle_gain_mm        = 7.0
circle_samples        = 36

# Chaining behavior
# If adjacent jittered points are farther than this, start a new polyline.
# Set to None to never split.
chain_split_gap_mm    = 9999.0   # safe default. Try 12.0 if your jitter is large.

# Drawing params
pos_off         = -0.10
settle_s        = 0.20
feed_grid       = 3000
feed_circles    = 1500

# Separate execution switches for different pens
execute_grid    = False
execute_circles = True

rng = random.Random(rng_seed)

# 3) Layout
W, H = (x1 - x0), (y1 - y0)
cx_page = x0 + 0.5*W
cy_page = y0 + 0.5*H

usable_w = W - 2*margin_mm
usable_h = H - 2*margin_mm
total_w = (n_cols - 1) * grid_dx_mm
total_h = (n_rows - 1) * grid_dy_mm
assert total_w <= usable_w + 1e-6 and total_h <= usable_h + 1e-6, \
    "Increase rectangle or reduce grid size or margins."

x_start = x0 + 0.5*(W - total_w)
y_start = y0 + 0.5*(H - total_h)

def center_weight(x: float, y: float, sigma: float) -> float:
    dx = x - cx_page
    dy = y - cy_page
    r2 = dx*dx + dy*dy
    return math.exp(-0.5 * r2 / (sigma*sigma)) if sigma > 1e-9 else 1.0

def jittered_point(x: float, y: float) -> XY:
    w = center_weight(x, y, jitter_falloff_sigma)
    sigma = jitter_sigma_center * w
    jx = rng.gauss(0.0, sigma)
    jy = rng.gauss(0.0, sigma)
    return (x + jx, y + jy)

def circle_poly(cx: float, cy: float, diam: float, samples: int) -> List[XY]:
    r = 0.5 * max(diam, 0.0)
    pts: List[XY] = []
    for i in range(samples):
        th = 2.0*math.pi * i / samples
        pts.append((cx + r*math.cos(th), cy + r*math.sin(th)))
    pts.append(pts[0])
    return pts

def chain_points(points: Iterable[XY], split_gap: float | None) -> List[List[XY]]:
    """
    Given an ordered sequence of points, return a list of polylines by
    splitting where adjacent points are far apart.
    """
    pts_list = list(points)
    if not pts_list:
        return []
    chains: List[List[XY]] = []
    cur: List[XY] = [pts_list[0]]
    for i in range(1, len(pts_list)):
        prev = pts_list[i-1]
        p = pts_list[i]
        if split_gap is not None and math.hypot(p[0]-prev[0], p[1]-prev[1]) > split_gap:
            if len(cur) >= 2:
                chains.append(cur)
            cur = [p]
        else:
            cur.append(p)
    if len(cur) >= 2:
        chains.append(cur)
    return chains

# 4) Build jittered grid points
grid_pts = [[(x_start + c*grid_dx_mm, y_start + r*grid_dy_mm)
             for c in range(n_cols)] for r in range(n_rows)]
jit_pts  = [[jittered_point(x, y) for (x, y) in row] for row in grid_pts]

# 5) Build strokes, separated into grid and circles
grid_strokes: List[List[XY]] = []
circle_strokes: List[List[XY]] = []

# Rows as long polylines
for r in range(n_rows):
    row_pts = jit_pts[r]  # left to right
    for chain in chain_points(row_pts, chain_split_gap_mm):
        grid_strokes.append(chain)

# Columns as long polylines
for c in range(n_cols):
    col_pts = [jit_pts[r][c] for r in range(n_rows)]  # top to bottom
    for chain in chain_points(col_pts, chain_split_gap_mm):
        grid_strokes.append(chain)

# Circles at nodes
for r in range(n_rows):
    for c in range(n_cols):
        x_nom, y_nom = grid_pts[r][c]
        xj, yj = jit_pts[r][c]
        w = center_weight(x_nom, y_nom, jitter_falloff_sigma)
        diam = circle_base_diam_mm + circle_gain_mm * w
        circle_strokes.append(circle_poly(xj, yj, diam, circle_samples))

def strokes_to_pattern(strokes: List[List[XY]], feed: float) -> Pattern:
    pat = Pattern()
    for pts in strokes:
        if len(pts) < 2:
            continue
        pl = Polyline(
            pts=pts,
            z_mode="midpoints",
            pos_offset=pos_off,
            settle_s=settle_s,
            min_seg_mm=0.15,
            max_seg_mm=1e9,
            feed_draw=feed,
        )
        setattr(pl, "_plot_color", "#000000")
        pat.add(pl)
    return pat

# 6) Build separate patterns
pat_grid    = strokes_to_pattern(grid_strokes,    feed_grid)
pat_circles = strokes_to_pattern(circle_strokes,  feed_circles)

# Combined preview
pat_preview = strokes_to_pattern(grid_strokes + circle_strokes, feed_grid)

title = (
    f"Gaussian jittered grid {n_cols}x{n_rows}, dx={grid_dx_mm} mm, dy={grid_dy_mm} mm, "
    f"jitter_center={jitter_sigma_center} mm, falloff={jitter_falloff_sigma} mm, "
    f"circle_base={circle_base_diam_mm} mm, circle_gain={circle_gain_mm} mm, "
    f"chain_split={chain_split_gap_mm} mm"
)
plot_plan((cfg.x_max, cfg.y_max), (x0, y0, x1, y1), [], pat_preview,
          step=50.0, figsize=(9.8, 6.6), title=title)

# 7) Optional execution by pen
if execute_grid:
    Renderer(grbl).run(pat_grid)
if execute_circles:
    Renderer(grbl).run(pat_circles)
# grbl.goto_abs(0, 0)


## 🧹 Cleanup

Return the pen to origin and close the serial connection when finished.

In [ ]:
grbl.goto_abs(0,0)

grbl.close()
print('Serial closed.')

In [ ]:
grbl.goto_abs(0,0)

In [ ]:
grbl.pen_down()